In [7]:
import json
from collections import OrderedDict, defaultdict
from operator import itemgetter

In [8]:
# Add BILOU and BIO representation.


In [9]:
building_name = 'ap_m'
with open('metadata/{0}_sentence_dict_justseparate.json'.format(building_name), 'r') as fp:
    word_sentence_dict = json.load(fp)
with open('metadata/{0}_label_dict.json'.format(building_name), 'r') as fp:
    word_label_dict = json.load(fp)

In [10]:
# These part could be automated with Brick.
category_dict = {
    'equipment': ['vav', 'chilled_water_pump', 'hot_water_pump', 'ahu', 'supply_fan', 'return_fan', 'exhaust_fan'],
    'network_adapter': ['vnd', 'network_adapter'],
    'idenfitier': ['identifier'],
    'location': ['room', 'floor', 'building', 'server'],
    'none': ['none']
}

In [11]:
def get_key(d, v):
    for k, v_list in d.items():
        if v in v_list:
            return k
    return 'pointtype'

In [12]:
char_sentence_category_dict = dict()
char_sentence_label_dict = dict()
for srcid, word_labels in word_label_dict.items():
    char_sentence_category = list()
    char_sentence_label = list()
    word_sentence = word_sentence_dict[srcid]
    for word, label in zip(word_sentence, word_labels):
        if label=='none':
            for char in word:
                char_sentence_label.append((char, 'O'))
                char_sentence_category.append((char, 'O'))
        else:
            category = get_key(category_dict, label)
            if len(word)==1:
                char_sentence_label.append((word[0], 'U_'+label))
                char_sentence_category.append((word[0], 'U_'+category))
            else:
                char_sentence_label.append((word[0], 'B_' + label))
                char_sentence_category.append((word[0], 'B_' + category))
                for char in word[1:-1]:
                    char_sentence_category.append((char, 'I_' + category))
                    char_sentence_label.append((char, 'I_' + label))
                char_sentence_label.append((word[-1], 'L_' + label))
                char_sentence_category.append((word[-1], 'L_' + category))
    char_sentence_category_dict[srcid] = char_sentence_category
    char_sentence_label_dict[srcid] = char_sentence_label

In [13]:
with open('metadata/{0}_char_category_dict.json'.format(building_name), 'w') as fp:
    json.dump(char_sentence_category_dict, fp, indent=2)
with open('metadata/{0}_char_label_dict.json'.format(building_name), 'w') as fp:
    json.dump(char_sentence_label_dict, fp, indent=2)